<h1>Kapitel 12: Objektorientierung, Mehrfachvererbung</h1>
<p>Grundlagen der Programmierung 1<p><h2>Holger Karl</h2>
 

<h1>Overview</h1> 

1. Überblick
1. Multiple inheritance
1. Method Resolution Order
1. Linearisierungsalgorithmus: C3
1. MRO, dependency injection  und super
1. Zusammenfassung 
 


<h1>Overview</h1> 

1. **<font color="red">Überblick</font>**
1. Multiple inheritance
1. Method Resolution Order
1. Linearisierungsalgorithmus: C3
1. MRO, dependency injection  und super
1. Zusammenfassung

# Setup

Main point here is to set up warnings properly for presentation and to
load tutormagic, so that we can later on use pythontutor for code
animations. 

In [ ]:
%load_ext tutormagic
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning, module='.*/IPython/.*')

# Was bisher geschah

-   Wir haben Grundlagen von Klassen und Objekten besprochen
-   Insbes. Objektorientierte Programmierung
    -   Kernkonzept: Vererbung

# Dieses Kapitel

-   Wir erweitern Vererbung auf die Möglichkeit, mehrere Oberklassen zur
    Vererbung zu nutzen
-   Dazu müssen wir festlegen, in welcher Reihenfolge Methoden der
    Oberklassen benutzt werden
-   Der Fall einer rautenförmigen Klassenhierarchie wird besonders
    interessant

<h1>Overview</h1> 

1. Überblick
1. **<font color="red">Multiple inheritance</font>**
1. Method Resolution Order
1. Linearisierungsalgorithmus: C3
1. MRO, dependency injection  und super
1. Zusammenfassung

# Grundidee

-   Vererbung diente der Code-Wiederverwendung und Spezialisierung
    -   Oberklasse stellt Code bereit; Unterklasse erweitert, überschreibt ggf.
-   Was, wenn Code aus unterschiedlichen Klassen benutzt werden soll?
-   Siehe auch [Python-Dokumentation](https://docs.python.org/3.5/tutorial/classes.html#inheritance)

# Beispiel Uhr, Kalender

-   Uhr: Klasse, die Uhrzeit bereit hält und auf `tick` eine Sekunde weiterzählt
    -   Um Mitternach auf 0:00:00 zurückspringt
-   Kalender: Klasse, die Datum bereit hält und auf `next_day` einen Tag weiterzählt
-   Siehe [Details](http://www.python-course.eu/python3_multiple_inheritance.php)

In [ ]:
class Clock:
    def __init__(self, hour, minute, second):
        # ...
    def tick(self):
        self.second += 1
        if self.second > 60:
            ...

class Calendar:
    def __init__(self, day, month, year):
        #...

    def next_day(self):
        self.day += 1
        if self.day > 28:
            # check month ...

# Beispiel: Uhr und Kalender?

-   Wie baut man eine *Uhr mit Kalender*?
    -   Von `Clock` ableiten, und die Kalender-Funktionen nachimplementieren?
    -   Umgekehrt?
-   Nicht schön!

# Mehrfache Oberklassen!

-   Natürliche Idee: Wir erben von zwei Klassen statt nur einer
    -   Können auf die Methoden dieser Klassen zugreifen, überschreiben, &#x2026;

<center>![img](./uml/mi-clock.png "Mehrere Oberklassen: Clock und Calendar")</center>

# Syntax?

-   Naheliegend: Einfache Vererbung mit Oberklasse in Klammer
-   Mehrere Oberklassen: Komma-separierte Liste der Oberklassen

In [ ]:
class C(Oberklasse1, Oberklasse2, ...):
    ...

# Semantik?

In diesem einfachen Beispiel ist die Semantik klar: 

-   Methodenaufruf: eindeutig zuzuordnen 
    -   Methodennamen der Oberklassen unterscheiden sich!
-   Datenzugriff: Passiert sowieso im Namensraum des Objekts

# Einfaches Beispiel

Schauen wir uns das mit einem einfachen Beispiel an

In [ ]:
class A():
    def m_a(self):
        print("a")

class B():
    def m_b(self):
        print("b")

class C(A, B):
    pass

c = C()
c.m_a()
c.m_b()

# Gleiche Methodennamen in Oberklassen?

Was passiert, wenn der gleiche Methodenname in mehreren Oberklassen auftritt? 

In [ ]:
class A():
    def m(self):
        print("a")

class B():
    def m(self):
        print("b")

# erst A, dann B: 
class C(A, B): pass
C().m()

# erst B, dann A:
class D(B, A): pass
D().m()

# Gleiche Methodennamen in Oberklassen &#x2013; Reihenfolge!

Offenbar ist die Reihenfolge der Oberklassen wichtig 

-   Vermutung: Methoden werden in den Oberklassen gesucht
    -   Liste von links nach rechts gelesen
-   Regel: Die Methode der zuerst angegebenen Klasse überdeckt die
    Methode der später angegebenen Klasse 
    -   (Da fehlen noch Details!)

# Insbesondere: `__init__` ?

Beispiel: `__init__` taucht in den meisten Klassen auf! 

-   Vereinbaren von Daten?

In [ ]:
%%tutor -t
class A():
    def __init__(self):
        self.a = 17

class B():
    def __init__(self):
        self.b = 42

# erst A, dann B: 
class C(A, B): pass

c = C()
print(c.a)
print(c.b)

# `__init__` typischerweise mit `super`?

Ist das Problem, dass wir in Klasse `C` keinen Konstruktor haben, der
den Konstruktor der Oberklasse aufruft? 

In [ ]:
%%tutor -t 
class A():
    def __init__(self):
        self.a = 17

class B():
    def __init__(self):
        self.b = 42

# erst A, dann B: 
class C(A, B):
    def __init__(self):
        super().__init__() 

c = C()
print(c.a)
print(c.b)

# `__init__` ruft Konstruktoren explizit auf?

Zur Erinnerung: Man kann Methoden der Klassen auch explizit aufrufen 

In [ ]:
class A():
    def __init__(self):
        self.a = 17

class B():
    def __init__(self):
        self.b = 42

# erst A, dann B: 
class C(A, B):
    def __init__(self):
        A.__init__(self)
        B.__init__(self)

c = C()
print(c.a)
print(c.b)

## Beobachtung<a id="orgheadline21"></a>

-   Das funktioniert!
-   Aber ist das schön? Verletzt DRY!
    -   Was, wenn man die Klassenhierarchie ändert und `C` von anderen
        Klassen ableitet?
    -   Und das nicht nur im Konstruktor benutzt wird?

# `super` im `__init__` der Oberklasse?

Was würde eigentlich passieren, wenn die Oberklasse im Konstruktur ebenfalls
`super` benutzt? 

-   Sie könnte ja ihrerseits abgeleitet sein
-   (Und das ist sie auch &#x2013; alle Klassen sind Unterklassen von
    `object`!)
-   In gewissem Sinne kanonischer Code:

In [ ]:
class A():
    def __init__(self):
        print("A init") 
        super().__init__() 
        self.a = 17

class B():
    def __init__(self):
        print("B init")
        super().__init__() 
        self.b = 42

# erst A, dann B: 
class C(A, B):
    def __init__(self):
        print("C init")
        super().__init__() 

c = C()
print(c.a)
print(c.b)

# Beobachtung

-   Entscheidend scheint der Aufruf von `super().__init__()` in Klasse A
    zu sein
-   Das hängt offenbar mit der Reihenfolge von `C(A,B)` vs. `C(B,A)`
      zusammen
-   Das müssen wir genauer anschauen! (Abschnitt Method Resolution
    Order; siehe unten)

# Gleiche Daten?

Kleine Variation zu oben: Beide Klassen benutzten gleiches
Datenattribut? 

-   Offenbar nur ein `x`
-   Das ist plausibel, wird zur Laufzeit angelegt!

In [ ]:
%%tutor -t 
class A():
    def __init__(self):
        print("A init") 
        super().__init__() 
        self.x = 17

class B():
    def __init__(self):
        print("B init") 
        super().__init__() 
        self.x = 42

# erst A, dann B: 
class C(A, B):
    def __init__(self):
        print("C init") 
        super().__init__() 

c = C()
print(c.x)

<h1>Overview</h1> 

1. Überblick
1. Multiple inheritance
1. **<font color="red">Method Resolution Order</font>**
1. Linearisierungsalgorithmus: C3
1. MRO, dependency injection  und super
1. Zusammenfassung


# Reihenfolge der Methodenaufrufe

Kommen wir auf dieses Beispiel zurück: 

In [ ]:
class A():
    def __init__(self):
        print("A init") 
        super().__init__() 

class B():
    def __init__(self):
        print("B init")
        super().__init__() 

# erst A, dann B: 
class C(A, B):
    def __init__(self):
        print("C init")
        super().__init__() 

c = C()

## Fragen: Reihenfolge<a id="orgheadline26"></a>

-   In welcher Reihenfolge werden die Methoden hier aufgerufen?
-   Was wäre wünschenswert?
-   Wie kann man das beeinflussen?
-   Gibt es relevante Sonderfälle?

-   Siehe [hier für Details](https://www.python.org/download/releases/2.3/mro/) (für Python 2.3 beschrieben; auch für Python
    3 gültig)

# Suche nach auszuführender Methode, erste Ideen

-   Finde Attribute spezialisierterer Klassen vor denen allgemeinerer Klassen
-   Zwischen gleichrangigen Klassen, lass den Programmierer der
    Unterklasse bestimmen
    -   Durch Angabe der Reihenfolge der Oberklasse für direkt Vorgänger
        schon erfüllt

# Suche nach auszuführender Methode, Sonderfall Einzelvererbung

Sonderfall (wie bisher): eine Klasse nur jeweils genau eine Oberklasse bis zu `object`

-   Also keine Mehrfachvererbung
-   Man sagt: *die Klassenhierarchie ist nicht **nach oben** verzweigt*

Dann ist die auszuführende Methode einfach zu finden: 

-   Suche in der Klasse selbst
-   Wenn nicht erfolgreich, suche in (eindeutig bestimmter) Oberklasse
-   Und rekursiv fortsetzen

Verallgemeinerung? 

# Linearisierung nötig für Suche

Wir brauchen: eine *Linearisierung* der Klassenhierachie 

-   Aus dem Graph der Oberklassen wird eine Liste der Klassennamen
-   Die Linearisierung bestimmt die Reihenfolge, in der nach Methoden
    gesucht wird
    -   Die *Method Resolution Order* (MRO)
-   Beispiel: Aus `class C(A, B)` wird die Linearisierung `C, A, B`

## Regel?<a id="orgheadline30"></a>

Was ist eine  **sinnvolle** Regel, um Linearisierung zu bestimmen? 

# Zugriff auf MRO

Python erlaubt unmittelbar Zugriff auf die MRO einer Klasse: Attribut
`__mro__` 

In [ ]:
class A(): pass
class B(A): pass
class C(B): pass 
print(C.__mro__)

# Anforderungen an sinnvolle Linearisierung: Monotonie

Angenommen: 

-   Wir betrachten die Linearisierung einer Klasse `C`
-   Darin kommt `C1` vor `C2`
-   Wir leiten von `C` eine neue Klasse `Cneu` ab
-   Was sollte dann in der Linearisierung von `Cneu` für die Reihenfolge
    von `C1` und `C2` gelten?

<div class="definition-head" style="border:3px; border-style:solid; border-color:#5da9e9; padding: 5px; background-color:#5da9e9;  line-height:1.5em; text-align:center; border-radius: 15px 15px 0px 0px; margin-bottom: 0; width: 100%;"><b>Definition: Monotonie<a id="orgheadline33"></a></b></div>

Kommt in der Linearisierung einer Klasse `C` die Klasse `C1` vor der
Klasse `C2`, so kommt `C1` auch vor `C2` in allen Linearisierungen von
Unterklassen von `C`. 

<div class="definition-content" style="border:3px; border-style:solid; border-color:#5da9e9; padding: 5px; margin-top: 0; border-radius: 0px 0px 15px 15px; width: 100%; text-align:left"></div>

## Warum?<a id="orgheadline34"></a>

Andernfalls: durch Ableiten einer Klasse von `C` könnte man die
Aufrufreihenfolge von Methoden verändern. Mit sehr subtilen Bugs. 

# Anforderungen an sinnvolle Linearisierung: Local precedence ordering

<div class="definition-head" style="border:3px; border-style:solid; border-color:#5da9e9; padding: 5px; background-color:#5da9e9;  line-height:1.5em; text-align:center; border-radius: 15px 15px 0px 0px; margin-bottom: 0; width: 100%;"><b>Definition: Local precedence ordering<a id="orgheadline36"></a></b></div>

Die Reihenfolge der direkten Oberklassen einer Klasse `C`  soll in der
Linearisierung der Klasse `C` respektiert werden. 

<div class="definition-content" style="border:3px; border-style:solid; border-color:#5da9e9; padding: 5px; margin-top: 0; border-radius: 0px 0px 15px 15px; width: 100%; text-align:left"></div>

## Warum?<a id="orgheadline37"></a>

Abweichendes Verhalten würde sicherlich jeden Programmierer
verwundern. 

# MRO: Erste Idee

-   Wir schauen von der Klasse nach oben
-   Wir suchen von links nach rechts in der Oberklassendeklaration
-   Wir durchsuchen die Oberklassen
    -   Option 1: Erst alle Eltern, dann die Großeltern, usw. (eine *Breitensuche*)
    -   Option 2: Erst die erste Oberklasse (der erste Elter), dann dessen
        Eltern, usw; dann die zweite Oberklasse (der zweite Elter),
        &#x2026; (eine *Tiefensuche*)
-   Besuchen wir eine Klasse zweimal? Nein

Beispiele? 

# Darstellung: einfachere Klassendiagramme

-   Zunächst: Wir vereinfachen die Klassendiagramme (UML braucht zu viel
    Platz)
-   Zwei äquivalente Darstellungen von  `class C(A,B)`

<center>![img](./uml/simpler-graph-0.png "Zwei äquivalente Darstellungen von `class C(A,B)`: UML")</center>

<center>![img](./uml/simpler-graph-1.png "Zwei äquivalente Darstellungen von `class C(A,B)`: Vereinfachtes Diagramm; Ziffern geben Reihenfolge der Oberklassen an")</center>

# Beispiel: Ein Klassendiamand

Anmerkung: 

-   Auf die Reihenfolge bei `A` und `B` achten!
-   `X(object)` ist nicht nötig; hier nur zur Verdeutlichung

<center>![img](./uml/diamond.png "Vereinfachtes Klassendiagramm für Diamanten")</center>

In [ ]:
class X(object): pass 
class Y(object): pass 
class A(X, Y): pass 
class B(Y, X): pass
print(A.__mro__) 
print(B.__mro__)

# Problem: Nicht alle Fälle linearisierbar

Es gibt Klassenhierarchien zu denen keine sinnvolle Linearsierung
 bestimmt werden kann

-   Beispiel: *Diamond of death*

In [ ]:
class X(object): pass
class Y(object): pass
class A(X,Y): pass
class B(Y,X): pass
class C(A, B): pass

## Diagramm<a id="orgheadline42"></a>

<center>![img](./uml/not-linearizable.png "Nicht linearisierbare Klassenhierarchie")</center>

# Problem: Nicht alle Fälle linearisierbar  (2)

Kern des Problems: Unterschiedliche Reihenfolgen von `X`, `Y` bei `A`,
`B`

In [ ]:
class X(object):
    def m(self): pass
class Y(object): 
    def m(self): pass

class A(X,Y):
    def a(self):
        m()
class B(Y,X):
    def b(self):
        m()

class C(A, B):
    def m(self):
        self.a()
        self.b()
        super().m()

# Uneinheitliche Reihenfolge

Was passiert bei: 

-   C.a(): Methode A.a() aufgerufen, die wird wohl X.m() aufrufen
-   C.b(): Methode B.b() aufgerufen, die wird wohl Y.m() aufrufen
    -   Also inkonsistentes Verhalten der Klasse `C` bzgl. Aufruf von `m`!
-   `super` in `C`: Welches `super.m` nehmen? `X.m`? `Y.m`?
    -   Unklar; beides ist plausibel wegen Verhalten bei `C.a`, `C.b`

## Optionen?<a id="orgheadline45"></a>

-   Option 1: Irgendeine Reihenfolge festlegen; Inkonsistenz akzeptieren
-   Option 2: Solche Klassenhierarchien nicht erlauben
    -   Python 3: Wirft Ecxeption beim Versuch, `class C` wie oben zu
        definieren

# Gesucht: Vorgehen für Linearisierung

Wir brauchen also: 

-   Einen Algorithmus, der eine sinnvolle Linearisierung bestimmt wenn
    möglich
    -   Also: Im Einklang mit Monotonie und local precedence
-   Oder einen Fehler mitteilt falls nicht möglich

<h1>Overview</h1> 

1. Überblick
1. Multiple inheritance
1. Method Resolution Order
1. **<font color="red">Linearisierungsalgorithmus: C3</font>**
1. MRO, dependency injection  und super
1. Zusammenfassung

# Linearisierungsalgorithmus: C3

-   Python benutzt den C3 Algorithmus
    -   Aus Sprache Dylan; auch in anderen OO-Sprachen benutzt
    -   [Quelle](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.19.3910&rep=rep1&type=pdf)

## Terminologie<a id="orgheadline48"></a>

-   Linearisierung einer Klasse (die MRO): eine geordnete Liste aller
    (direkten oder indirekten) Oberklassen der Klasse
    -   Notation: C1 C2 &#x2026; CN
-   *Head* einer Liste `L`: das erste Element, `L[0]`
-   *Tail* einer Liste `L`: der Rest der Liste ab zweitem Element,
    `L[1:]`, ggf `None`
-   *Verkettung*: als + geschrieben

# C3: Grundidee

Die Linearisierung `Lin` einer Klasse `C` ist 

-   die Klasse `C` selbst,
-   verkettet mit der Kombination (*merge*)
    -   der Linearisierung seiner
        direkten Oberklassen und
    -   der Liste der direkten Oberklassen (in Reihenfolge wie angegeben)
        -   Um local precedence sicherzustellen
-   Rekursiver Algorithmus!
    -   Basisfall: `Lin(object) = object`

Oder kurz: 

`Lin(C(B1, B2, ..., Bn)) = C + merge(Lin(B1), Lin(B2), ..., Lin(Bn),
[B1, B2, ... Bn]))`

# C3: Merge

Was bedeutet *merge*?

-   Funktion merge durchsucht Oberklassen, sucht nach  Konflikten
-   merge arbeitet auf mehreren Linearisierungen, produziert daraus eine
    (oder Fehler)

## Gewünschtes Verhalten: Klasse von `object` abgeleitet<a id="orgheadline51"></a>

`Lin(C(object)) = C + merge(Lin(object), [object])`

-   `= C + merge([object], [object])`
-   `= C + object`

# C3: Merge &#x2013; *good heads*

Wir brauchen das Konzept eines *good heads* um Merge zu formulieren 

-   Gegeben:
    -   Eine Liste von Linearisierung: [Lin1, Lin2, &#x2026; Lin n]
        -   Jede Linearisierung ist eine Liste von Klassen
        -   Leere Linearisierungen werden ignoriert
-   Gesucht: Ist der Head einer  Liste *good* ?
    -   Ein Head ist good wenn er in keinem Tail irgendeiner Liste vorkommt
        -   Ein good head darf head mehrerer Listen sein

## Beispiel<a id="orgheadline53"></a>

-   [ABC, BCD, BCA] &#x2013; weder A noch B sind good heads
-   [ABC, BCD, BCF] &#x2013; A ist good, B nicht

# C3: Merge &#x2013; Definition

-   `merge (Lin1, ... Lin n) = X  + merge(Lin1 - X, ... Lin n - X)`
      falls X der **erste** good head in Lin1, &#x2026;, Lin n ist
    -   `merge( [], ... , []) = []`
    -   Notation `l - X`: die Liste `l` aus der der Wert `X` entfernt wurde
-   Nicht definiert (Fehler), falls es keinen good head gibt

# Beispiele

-   `merge([ABC, BCD, BCA]) = Fehler`
    -   Fehler: Weder A noch B sind good heads
-   `merge([ABC, BCD, BCF]) = A + merge([BC, BCD, BCF]) = A + B +  merge([C, CD, CF]) = A + B + C + merge([ [] , D, F]) = A + B + C +  D + F`
    -   Leere Linearisierungen werden ignoriert
-   `merge([ABC, BCD, FCB]) = A + merge([BC, BCD, FCB]) = A +  F +  merge([BC, BCD, CB]) = A + F + Fehler = Fehler`
    -   Fehler tritt erst später auf; insgesamt aber doch Fehler
-   `merge([AB, CB, DB]) = A + merge([B, CB, DB]) = A + C + merge([ B,  B,  DB]) = A + C + D + merge(B, B, B) = A + C + D + B`
    -   `B` ist erst im letzten Schritt ein good head
-   `merge(ABEF, CD, BEF) = A + B + E + F + C + D`
    -   Wir fangen jeweils wieder vorne an! Tiefensuche!

# C3: Klassendiagramm-Beispiel 0: Keine überlappenden Oberklassen

<center>![img](./uml/mro0-0.png "Einfachstes Beispiel für C3-Linearisierung: Keine überlappenden Oberklassen")</center>

In [ ]:
class D(object): pass 
class E(object): pass 
class F(object): pass 
class G(object): pass 
class B(D,E): pass 
class C(F,G): pass
class A(B,C): pass
print(B.__mro__) 
print(C.__mro__) 
print(A.__mro__)

## Beobachtung<a id="orgheadline57"></a>

-   Keine überlappenden Oberklassen (außer `object`)
-   Beobachtung: Tiefensuche!

# C3: Klassendiagramm-Beispiel 0  (2)

Linearisierung (O kurz für object): 

-   Trivial: Lin(D)=DO, Lin(E)=EO, Lin(F)=FO, Lin(G)=GO
-   Lin(B) = B + merge(DO, EO, DO) = B + D + merge(O, EO, O) = B + D +
    E + merge(O, O, O) = BDEO
-   Lin(C) = CFGO entsprechend
-   Lin(A) = A + merge(BDEO, CFGO, BC)
    -   = A + B + merge(DEO, CFGO, C)
    -   = A + B + D +  merge(EO, CFGO, C)
        -   Wir fangen beim nächsten merge-Schritt wieder mit dem ersten
            Argument an, falls möglich
        -   Das führt zu Tiefensuche
    -   = A + B + D + E + merge(O, CFGO, C)
        -   O wird aufgeschoben (nicht good), da es Oberklasse von C ist; Linearisierung 
            von C muss erst eingebaut werden
    -   = A + BDE + CFG + merge(O, O, []) = ABDECFGO

# C3: Klassendiagramm-Beispiel 0  (3)

Pfeil: Reihenfolge der Methodensuche ab `A` 

<center>![img](./uml/mro0-1.png "Triviales Beispiel für C3-Linearisierung mit MRO")</center>

# Beobachtung: Teilsequenz

-   Die Linearisierungen von `B` und `C` sind als Teilsequenzen in der
    Linearisierung von `A` enthalten
-   Ist das immer so?

Nein &#x2013; siehe folgende Beispiele! 

# C3: Klassendiagramm-Beispiel 1

Ziffern: Reihenfolge der Oberklassen

<center>![img](./uml/mro1-0.png "Erstes Beispiel für C3-Linearisierung")</center>

In [ ]:
class D(object): pass 
class E(object): pass 
class F(object): pass
class B(D,E): pass 
class C(D,F): pass
class A(B,C): pass
print(B.__mro__) 
print(C.__mro__) 
print(A.__mro__)

# C3: Klassendiagramm-Beispiel 1  (2)

Linearisierung (O kurz für object): 

-   Trivial: Lin(D) = DO, Lin(E) = EO, Lin(F) = FO
-   Lin(B) = B + merge(DO, EO, DE) = B + D + merge(O, EO, E) = B + D +
    E + merge(O, O, O) = BDEO
-   Lin(C) = CDFO (analog)

## Interessant: Lin(A)<a id="orgheadline63"></a>

-   Lin(A) = A + merge(Lin(B), Lin(C), BC)
    -   = A + merge(BDEO, CDFO, BC)
    -   = A + B + merge(DEO, CDFO, C)
    -   = A + B + C + merge(DEO, DFO, [])
    -   = A + B + C + D + merge(EO, FO, [])
    -   = ABCDEFO

# C3: Klassendiagramm-Beispiel 1  (3)

Grüner Pfeil: Reihenfolge der Methodensuche ab `A` 

<center>![img](./uml/mro1-1.png "Erstes Beispiel für C3-Linearisierung mit MRO")</center>

# C3: Klassendiagramm-Beispiel 2

Kleine Änderung: Reihenfolge bei Klasse `B` vertauscht 

<center>![img](./uml/mro2-0.png "Zweites Beispiel für C3-Linearisierung")</center>

In [ ]:
class F(object): pass
class E(object): pass
class D(object): pass
class C(D,F): pass
class B(E,D): pass
class A(B,C): pass
print(B.__mro__) 
print(C.__mro__) 
print(A.__mro__)

# C3: Klassendiagramm-Beispiel 2

<center>![img](./uml/mro2-1.png "Zweites Beispiel für C3-Linearisierung mit MRO für A, B, C")</center>

# Praktische Konsequenz?

<div class="alertblock-head" style="border:3px; border-style:solid; border-color:#f96900; padding: 5px; background-color:#f96900;  line-height:1.5em; text-align:center; border-radius: 15px 15px 0px 0px; margin-bottom: 0; width: 100%;"><b>Achtung! Reihenfolge der Oberklasse<a id="orgheadline68"></a></b></div>

Achten Sie auf Reihenfolge der Oberklassen!!

Ein unachtsames Umdrehen führt zum Ausführen anderer Methoden &#x2013; in
diesen Beispielen würde statt einer Methode `D.m` eine Methode
gleichen Namens `E.m` ausgeführt! 

<div class="alertblock-content" style="border:3px; border-style:solid; border-color:#f96900; padding: 5px; margin-top: 0;  line-height:1.5em; border-radius: 0px 0px 15px 15px; width: 100%; text-align:left"></div> 

# C3: Nicht-linearisierbare Hierarchie

Erinnerung: Dieses Beispiel nicht linearisierbar

In [ ]:
class X(object): pass
class Y(object): pass
class A(X,Y): pass
class B(Y,X): pass
class C(A, B): pass

## Verhalten von C3?<a id="orgheadline70"></a>

Lin(C) = C + merge(Lin(A), Lin(B), AB))

-   = C + merge(AXYO, BYXO, AB)
-   = C + A + merge(XYO, BYXO, B)
-   = C + A + B + merge(XYO, YXO, [])

Abbruch, da weder X noch Y ein *good head* sind &#x2013; zirkuläre Struktur 

<h1>Overview</h1> 

1. Überblick
1. Multiple inheritance
1. Method Resolution Order
1. Linearisierungsalgorithmus: C3
1. **<font color="red">MRO, dependency injection  und super</font>**
1. Zusammenfassung

# MRO und Methodenaufrufe in Oberklassen

Schauen wir uns dieses Beispiel genauer an (vereinfachte Version von
Beispiel 2 oben): 

In [ ]:
class D: pass
class E:
    def m(self):
        print("m in E")
class B(D, E):
    def f(self):
        print("f in B, calling m")
        self.m()

class C(E):
    def m(self):
        print("m in C")

class A(B, C): pass

# Ausgabe: m in D (klar)    
print(B.__mro__)
b = B()
b.f()

# Ausgabe hier?     
print(A.__mro__)
a = A()
a.f()

# MRO und Methodenaufrufe in Oberklassen &#x2013; Veranschaulichung

<center>![img](./uml/depend-inject.png "Methodenaufrufe aus Oberklassen: MRO")</center>

# MRO und Methodenaufrufe in Oberklassen &#x2013; Analyse `b.f()`

Was passiert bei den beiden Aufrufen von `f`? 

-   Bei Aufruf `b.f()`:
    -   `b` (referenziert unter dem Namen  `self` in den Methoden) ist ein Objekt der Klasse `B`
    -   Also wird die MRO dieser Klasse benutzt, um das richtige `f` zu
        finden
    -   MRO: BDEO
    -   Unter diesen Klassen gibt es ein `f` nur in Klasse `B`; dieses `f`  wird ausgeführt
    -   `f` ruft dann `m` auf
    -   Wieder wird MRO: BDEO benutzt
    -   Unter diesen Klassen gibt es ein `m` nur in Klasse `E`; dieses `m`  wird ausgeführt

# MRO und Methodenaufrufe in Oberklassen &#x2013; Analyse `a.f()`

-   Bei Aufruf `a.f()`:
    -   `a` (referenziert unter dem Namen  `self` in den Methoden) ist ein Objekt der Klasse `A`
    -   Also wird die MRO dieser Klasse benutzt, um das richtige `f` zu
        finden
    -   MRO: ABDCEO
    -   Unter diesen Klassen gibt es ein `f` nur in Klasse `B`; dieses `f` wird ausgeführt
    -   `f` ruft dann `m` auf
    -   Welches `m` ist jetzt gemeint?
        -   Es gibt `m` in `E` und `C`
    -   **Entscheidender Punkt**: Bei der Suche nach `m` wird das MRO von
        Klasse `A` benutzt!
        -   Obwohl `f` eine Methode der Klasse `B` ist
        -   Aber das ist egal &#x2013; es geht um die MRO der Klasse des
            *Objektes*, nicht die MRO der Klasse der gerade ausgeführten
            Methode
    -   Also nach MRO: ABDCEO wird eine Methode `m` zuerst in `C` gefunden
    -   Also wird `C.m` ausgeführt!

# MRO: Klasse des Objektes entscheidend

<div class="alertblock-head" style="border:3px; border-style:solid; border-color:#f96900; padding: 5px; background-color:#f96900;  line-height:1.5em; text-align:center; border-radius: 15px 15px 0px 0px; margin-bottom: 0; width: 100%;"><b>Achtung! Klasse des Objektes entscheidet über MRO<a id="orgheadline76"></a></b></div>

Bei einem Methodenaufruf entscheidet die Klasse des Objektes, welche
MRO benutzt wird. Die Klasse der dabei gerade aufgeführten Methode ist
egal. 

<div class="alertblock-content" style="border:3px; border-style:solid; border-color:#f96900; padding: 5px; margin-top: 0;  line-height:1.5em; border-radius: 0px 0px 15px 15px; width: 100%; text-align:left"></div> 

# MRO: Nutzung von `super`?

Was passiert, wenn in einer Methode die Funktion `super` aufgerufen
wird? 

-   Beispiel: `super().m()`
-   Bisherige Intention: `super` liefert die Oberklasse, ruft dort die
    Methode `m` auf

## Intuition passt nicht!<a id="orgheadline78"></a>

-   Diese Intuition war korrekt im Fall der Einfachvererbung
-   Bei Mehrfachvererbung: *die* Oberklasse ist nicht mehr eindeutig
    bestimmt
-   Wir brauchen präzisere Definition für `super()`!

# `super` bei Mehrfachvererbung: MRO

-   `super` sucht nach einer auszuführenden Methode
-   Für diese Suche haben wir ja die MRO definiert

<div class="definition-head" style="border:3px; border-style:solid; border-color:#5da9e9; padding: 5px; background-color:#5da9e9;  line-height:1.5em; text-align:center; border-radius: 15px 15px 0px 0px; margin-bottom: 0; width: 100%;"><b>Definition: `super` und MRO<a id="orgheadline80"></a></b></div>

Sei `c` ein Objekt einer Klasse `C`, die Klasse `B` eine (direkte oder
indirekte) Oberklasse von `C` (oder auch `C` selbst). 

Wird die Funktion `super` bei der Ausführung einer Methode `m1` der
Klasse `B` aufgerufen um eine Methode `m2` aufzurufen, so wird `m2` in der
MRO **von `C`** gesucht, **ab Klasse `B`**! 

<div class="definition-content" style="border:3px; border-style:solid; border-color:#5da9e9; padding: 5px; margin-top: 0; border-radius: 0px 0px 15px 15px; width: 100%; text-align:left"></div>

# `super` und MRO: Beispiel 1

Achten Sie auf den Unterschied der Ausgabe in den beiden Aufrufen von
`m` in `D.f`

In [ ]:
class B:
    def m(self):
        print("m in B")
class C:
    def m(self):
        print("m in C")

class D(C):
    def f(self):
        # Auf diesen Unterschied achten: 
        print("f in D, calling m via super: ") 
        super().m()
        print("f in D, calling m via self: ") 
        self.m()

class A(B, D): 
    def f(self):
        print("f in A, calling super")
        super().f()

# Ausgabe hier?     
print(A.__mro__)
a = A()
a.f()

# `super` und MRO: Beispiel 2

-   MRO von `A`: ABDCO
-   Aufruf von `super().m()` in `B.m` führt zum Aufruf einer *tieferen*
      Klasse als `B` selbst!
    -   Keineswegs eine Oberklasse!

In [ ]:
class O:
    def m(self):
        print("I am the ultimate m")
class B(O):
    def m(self):
        print("m in B")
        super().m() 
class C(O):
    def m(self):
        print("m in C")
        super().m() 

class D(C):
    def m(self):
        # Auf diesen Unterschied achten: 
        print("f in D, calling super") 
        super().m()

class A(B, D): 
    def m(self):
        print("m in A, calling super")
        super().m()

# Ausgabe hier?     
print(A.__mro__)
a = A()
a.m()

# `super` und MRO: Beispiel `__init__`

Wie oft werden die Konstruktoren hier ausgeführt?

-   Warum?
-   Ist das so gewollt?

In [ ]:
class O: 
    def __init__(self):
        print("O")
        super().__init__()

class B(O):
    def __init__(self):
        print("B")
        super().__init__()

class C(O):
    def __init__(self):
        print("C")
        super().__init__()

class A(B,C):
    def __init__(self):
        print("C")
        super().__init__()

a = A()

# `super`: Terminologie

-   Angesichts dieser Beispiele (und Definition) ist der Name `super` unglücklich
    -   Es wird eben nicht zwangsläufig eine Oberklasse aufgerufen; es
        kann zur Ausführung von Geschwisterklassen kommen
-   Besser wäre: `next_method()` oder ähnlich
    -   Auch, um Unterschied zu anderen OO-Sprachen deutlich zu machen,
        die in der Tat nur ein *primitives* super besitzen
        -   Und deswegen den Diamond of Death nicht vernünftig behandeln
            können!
-   Siehe auch  [super considered super](https://rhettinger.wordpress.com/2011/05/26/super-considered-super/)

# Cooperative multiple inheritance

MRO und die konsistente Nutzung von `super` in einer Klassenhierarchie
führen zu Python-typischem Programmiermodell: 

-   Explizite Referenzen auf Namen von Oberklassen sind nicht sinnvoll;
    `super` nutzen
-   Abgeleitete Klassen rufen (in aller Regel) in überschriebenen
    Methoden die überschriebene Methode mit  `super()`  auf
    -   Bis auf eine weit oben stehende Basisklasse, in der die
        `super`-Folge terminiert
-   Argumente müssen übereinstimmen
    -   Typisches Beispiel: Konstruktoren
    -   Nutze `*args` und `**kwargs`!

Insgesamt: Klassen sind auf den Aufruf durch `super` vorbereitet und
geben ihn entsprechend weiter 

-   Sie kooperieren miteinander: *cooperative multiple inheritance*
-   (We are all adults&#x2026;)

# Dependency injection

-   Cooperative multiple inheritance erlaubt *cool stuff*
-   Beispiel: dependency injection
    -   Wir ändern das Aufrufverhalten  durch Ableiten!

# Dependency injection: Pizzeria-Beispiel

-   Siehe: [Talk @ PyCon 2015, Youtube](https://www.youtube.com/watch?v=EiOglTERPEo)

In [ ]:
class Teigfabrik:
    def get_teig(self):
        print("Teig aus genmanipuliertem Weizen") 

class Pizzeria(Teigfabrik):
    def order_pizza(self):
        teig = super().get_teig()

Pizzeria().order_pizza()

# Dependency injection: Pizzeria-Beispiel (2)

In [ ]:
class Teigfabrik:
    def get_teig(self):
        print("Teig aus genmanipuliertem Weizen") 

class Pizzeria(Teigfabrik):
    def order_pizza(self):
        teig = super().get_teig()

class BioTeigfabrik(Teigfabrik):
    def get_teig(self):
        print("Biologischer Weizen führt zu Teig")

class BioPizzeria(Pizzeria, BioTeigfabrik): pass

BioPizzeria().order_pizza()
print(BioPizzeria.__mro__)

# Dependency injection: Muster

-   Pizzeria-Beispiel zeigt typisches Muster:
    -   Durch geschicktes Ableiten  von zwei Klassen wurde Funktionalität
        vereinigt
    -   Und Funktionalität einer Oberklasse ersetzt!
    -   Leere Unterklasse: typisch!
-   Kommt häufig in größeren Python-Programmen vor
-   Mächtig, flexibel, deterministisch
    -   MRO wurde für solche Zwecke gebaut!
    -   Aber auch mit Vorsicht zu benutzen!

# Einfachere Variante: Mixins

-   Gleiche Grundidee, aber etwas einfacher: Mixin
-   Mixin: Prima für Reuse fertigen Codes, nicht zur Spezialisierung
-   Vorgehen:
    -   Mixin-Klasse stellt Methoden zur Verfügung
    -   Klasse erbt (u.a.) von einer Mixin-Klasse
    -   &#x2026; und nutzt die Methoden des Mixins,
    -   verändert diese aber typischerweise nicht

# Mixin: Muster

-   Typische Muster für Mixins:
    -   Basisklasse und Mixin-Klasse(n) zusammen entworfen
    -   Basisklasse implementiert Methoden der Mixins als leere Methoden
    -   Eigentlich Klasse: erbe von Basisklasse und gewünschtem Mixin
-   Beispiel
    -   Webframeworks ([Django](https://docs.djangoproject.com/en/1.10/topics/class-based-views/mixins/)):
        -   Code zum Anzeigen eines einzelnen Objekts oder
            Liste von Objekten als Mixin
        -   Eigene View-Klasse erbt u.a. das gewünschte Mixin

<h1>Overview</h1> 

1. Überblick
1. Multiple inheritance
1. Method Resolution Order
1. Linearisierungsalgorithmus: C3
1. MRO, dependency injection  und super
1. **<font color="red">Zusammenfassung</font>**

# Zusammenfassung

-   Mehrfachvererbung (multiple inheritance) ist ein mächtiger, nahezu
    unverzichtbarer Bestandteil von OO
-   Es gibt leicht zu beherrschende Fälle: Oberklassen sind disjunkt
-   Es gibt Fälle, die Überlegung brauchen: Oberklassen definieren die
    gleichen Methoden
    -   *Diamond of Death*
    -   Mit geeignetem Linearisierungsalgorithmus für MRO aber problemlos
        zu lösen!
-   Python-spezifische Aspekte (Funktion von `super`) erlauben
    zusätzliche mächtige Ansätze wie dependency injection

  